# 라이브러리

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
#템플릿 생성 - 채팅 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
import getpass
import os
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [9]:
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4o")



 ········


## 지연/파업 템플릿

In [10]:
template=pd.read_csv("template.csv")
delay_template=template.iloc[0,2]
strike_template=template.iloc[1,2]

In [ ]:
시간표변경 합칠때 쓰세용
pd.concat([template,시간표변경.csv]).to_csv("template.csv")

# Text_generation 함수

In [1]:
def text_generation(source_content,category):
    #지연
    if category=="delay":
        prompt_template=delay_template
        chain_lst=["지연/사고 일시","지연/사고 노선","지연/사고 이유"]
    #파업
    elif category=="strike":
        prompt_template=strike_template
        chain_lst=["파업 일시","파업 노선","파업 이유"]
    #시간표변경
    else:
        pass
    prompt = ChatPromptTemplate(
    template=prompt_template,
    messages=[
    SystemMessagePromptTemplate.from_template(
    """Objective:
    Learn subway information through chaining and use that information to write a coherent and informative blog post using a template.
    Basic Setting
    Your name is Jitong.
    You are a subway information supporter and a Naver Blog Power Blogger.
    You are well-informed about information and value communication with readers.
    Your goal is to help subway users by providing accurate and practical information for convenient and safe subway use.
    
    Features and Activities
    You read the latest news related to subway """+ category+""", and write blog posts to accurately deliver these in a readable and high-quality manner to readers.
    You alleviate readers' inconveniences and speak empathetically through the blog, understanding the sentiments of subway users.
    
    Communication Style
    You use professional yet warm and easy-to-understand language.
    You emphasize the ability to explain things in a way that is accessible to all age groups.
    Your blog posts should end with the forms -어요, -이에요/예요, -(이)여요, -(이)요.
    
    Hallucination
    You always generate blog posts based on verifiable factual statements.
    You speak mainly about factual information related to subways and do not add information about subways on your own.
    """
    ),
    # The `variable_name` here is what must align with memory
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")
    ]
    )
    
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )
    #정보학습
    conversation({"question":"subway strike information(article) :"+source_content+ " Just REVIEW subway " +category+ " information"})
    #정보학습 - 체이닝 : chain_lst -> 일시, 노선, 이유
    for i in chain_lst:
        r=conversation({"question":"Using the provided information \n write "+i+":"})
    #글작성 - 체이닝 -> 정확성
    conversation({"question":"블로그 글 작성해줘"})
    conversation({"question":"1. 뉴스기사의 내용을 학습해 2. 학습한 뉴스기사와 블로그글을 비교해 3.블로그 글에 틀린 정보가 있다면 수정해 뉴스기사: "+source_content+"블로그 글 :"})
    #글 작성 - 템플릿
    result=conversation({"question":prompt_template})
    memory.clear()
    return result["text"]

### 템플릿 평가

In [14]:
df=pd.read_csv("GEVAL.csv",index_col="Unnamed: 0")
eval=[df.iloc[0,0],df.iloc[1,0],df.iloc[2,0],df.iloc[3,0],df.iloc[4,0]]

In [7]:
def evaluate_blog(blog_post, evaluation_prompt, source=None):

    evaluation_result = ""

    # 정확성 이외 평가 (source 필요 없음)
    if source==None:
        prompt = ChatPromptTemplate.from_messages([
        ("system", evaluation_prompt),
        ("user", "{input}")
        ])
        chain=prompt|llm|output_parser
        evaluation_result = chain.invoke({"input": f"blog content: {blog_post}"})


    # 정확성 평가 (source 필요)
    if source is not None:
        prompt = ChatPromptTemplate.from_messages([
        ("system", evaluation_prompt),
        ("user", "{input}")
        ])
        chain = prompt | llm | output_parser
        evaluation_result=chain.invoke({"input": "blog content:"+blog_post+"article, source:"+source})
    return evaluation_result

## 기사정보

파업

In [107]:
source_link=["https://www.joongang.co.kr/article/25192582#home","https://www.khan.co.kr/national/national-general/article/202311091911001","https://www.hani.co.kr/arti/society/rights/1115544.html","https://www.joongang.co.kr/article/25131013#home","https://www.sedaily.com/NewsView/29X8F079IZ"]


In [108]:
source=["""“파업소식에 1시간 일찍 출근”…기차 취소돼 급히 버스 예매도
중앙일보
수정 2023-09-15 00:43:28

철도노조 서울지방본부 총파업 출정식이 14일 오후 서울역 3번 출구 인근에서 열리고 있다. 이들은 공공철도 확대와 4조 2교대 전면 확대, KTX 수서역 운행 등을 요구하고 있다. 코레일은 파업에 따라 KTX와 새마을호, 무궁화호 등 여객열차와 화물열차를 20~60% 감축 운행에 들어갔다. [연합뉴스]

전국철도노동조합(이하 철도노조)이 14일 오전 9시부터 총파업에 들어갔다. 코레일은 파업 여파로 열차를 감축 운행했다. KTX 운행률은 평소의 76.4%였다. 중소도시를 연결하는 일반 여객열차(새마을호·무궁화호) 운행률은 평소의 68.1%로 떨어져, 이를 주로 이용하는 승객이 불편을 겪었다. 화물열차 운행률도 평소의 26.3%로 떨어졌다. 코레일은 수출입 화물과 산업 필수품 등 긴급 화물 위주로 화물열차를 투입했다.

오는 18일까지로 예정된 철도노조 파업은 2019년 11월 이후 4년 만이다. 철도노조는 ▶공공철도 확대 ▶4조 2교대 전면 확대 ▶KTX 수서역 운행 등을 요구하고 있다. 코레일은 KTX 수서역 운행 등은 정부 정책 사항으로 노사 교섭의 대상이 아니라는 입장이다.

관련기사
[사진] 철도파업 … 출근길 전철 간격 늘어나고 화물열차 운행률 26%
급작스러운 열차 운행 취소로 승객 불편이 불가피했다. 대전시 유성구에 사는 대학생 정모(21)씨는 “16일 서울에서 열리는 행사에 참석하기 위해 예매한 기차표가 취소돼 급히 버스를 예매했다”고 말했다. 부산역의 경우 KTX는 106회에서 74회로, 새마을호는 20회에서 16회, 무궁화호는 36회에서 25회로 운행이 줄었다. 사업차 매주 수도권을 찾는 우모(53)씨는 “앱으로 확인해 보니 파업 기간 부산에서 서울을 오가는 기차편 상당수가 운행 중지됐거나 매진됐다”며 “다른 이동수단을 찾는 중”이라고 말했다.

이날 인천 부평역에서 시민들이 열차를 기다리는 모습. [뉴스1]

파업 시작과 함께 수도권 전철 운행률도 평소의 83%로 떨어졌다. 파업 첫날 출근길 불편은 우려한 만큼 크지 않았지만 일부 열차의 지연 등으로 혼란이 빚어졌다. 지하철 1·2호선 환승역인 시청역에서 출근 시간에 1호선 열차가 지연됐다. 연착한 열차의 승객이 내리자마자 뛰어가는 모습도 눈에 띄었다. 직장인 유모(37)씨는 “유연근무제를 활용해 오전 10시 출근인데, 파업 소식에 한 시간 일찍 나왔다”고 말했다. 파업 사실을 모른 채 나왔다가 어리둥절해하기도 했다. 서울 은평구에서 종로구로 출근하는 김모(41)씨는 “평소 4~5분이던 1호선 배차 간격이 10분이 넘어 의아했다가 파업을 한다는 전광판 공지를 보고서야 알았다”고 말했다.

파업이 이어질 18일까지 다른 교통수단을 알아보는 경우도 있었다. 직장이 시청 근처인 이유진(30)씨는 “퇴근길 상황을 보고 내일(15일)은 버스를 이용할지 고민하겠다”고 말했다. 수도권 지방자치단체들은 별도 수송 대책을 마련했다. 경기도는 11개 노선에 예비차 98대를 증차하고, 필요할 경우 전세버스도 투입할 계획이다. 또 출근 시간에 광역버스 59대(80회 증회), 시내버스 18대(130회 증회)를 추가 배치한다. 인천시는 15·18일 출근 시간대에 지하철 7호선 상·하선 운행을 2회씩 증회하고, 경인선 주안·동암·부평역과 수인선 주요 환승역, 이용객이 많은 일부 역에 전세버스를 투입하기로 했다.

화물열차 운행은 크게 줄었다. 부산신항역과 경기도 의왕 오봉역을 오가는 화물열차는 평소 13회에서 이날 5회로 급감했다. 부산신항역 하루 운송량은 평균 1000TEU(1TEU는 20피트 컨테이너 1개)가 넘는데, 이날은 절반에도 미치지 못했다. 강원·충북 지역에 밀집한 시멘트업체는 납품 차질을 우려하고 있다. 산업통상자원부는 파업이 수출에 미칠 영향을 최소화할 수 있도록 모니터링을 강화하고, 비상 수송 대책을 추진하기로 했다.""",
"""
“걱정돼서 일찍 나왔다”…지하철 경고 파업 첫날, 퇴근길 혼잡도↑
2023.11.09 19:11 입력 2023.11.10 11:04 수정
임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 첫날인 9일 서울 지하철 5호선 광화문역이 퇴근하는 시민들로 붐비고 있다. 2023.11.09. 권도현 기자
임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 첫날인 9일 서울 지하철 5호선 광화문역이 퇴근하는 시민들로 붐비고 있다. 2023.11.09. 권도현 기자

“교통공사 노조 파업으로 인해 열차 운행 지연될 수 있습니다. 장시간 지연 시 다른 교통수단을 이용해 주시기 바랍니다.”

서울 지하철 1~8호선을 운영하는 서울교통공사 노조의 경고 파업이 시작된 9일, 서울 전역 지하철 역사에서 이같은 안내 방송이 종일 흘러나왔다. 지하철 개찰구와 승강장 곳곳에는 “안전인력 감축, 시민·노동자 안전은 누가 지킵니까” “시민 부담 전가, 노동 안전 파괴, 즉각 중단하라” 등이 적힌 노조 홍보물이 붙었다. 오후 6시쯤 5호선 서대문역에서 만난 원상훈씨(55)는 “지하철 파업 소식을 듣고 회사에서 일찍 출발했다”면서 “걱정돼서 전철역까지 7분 거리를 내내 뛰었다”고 말했다.

서울교통공사의 민주노총 소속 노동조합이 이날부터 이틀간 경고파업에 돌입하면서 첫날 퇴근길에 일부 혼잡이 발생했다. 오전 7~9시 출근 시간대에는 지하철이 정상 운행됐으나 퇴근 시간인 오후 6시 이후에는 배차 간격이 다소 늘거나 평소 수준으로 유지됐다. 일찍 퇴근길에 나선 직장인들이 오후 5시부터 몰리면서 역사 내부가 비교적 이른 시간에 빽빽이 들어찼다. 전철을 피해 버스를 타려는 시민들로 버스 정류장에 긴 대기줄이 이어지기도 했다.

임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 9일 5호선 광화문역에 관련 내용이 적힌 포스터가 붙어 있다. 2023.11.09 성동훈 기자
임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 9일 5호선 광화문역에 관련 내용이 적힌 포스터가 붙어 있다. 2023.11.09 성동훈 기자

이날 오후 5시쯤부터 지하철역은 이른 퇴근길에 나선 이들로 붐볐다. 오후 5시30분쯤 5호선 서대문역에서 만난 김모씨(47)는 “뉴스를 보고 지하철 (노동자들이) 파업한다는 사실을 알았다”면서 “공덕까지 가야 하는데 10분이 지나도 지하철이 안 왔다”고 말했다. 김씨는 “이미 교통카드를 찍고 개찰구 안으로 들어온 이상 그냥 기다렸다 가려고 한다”고 했다. 2호선 성수역에서 만난 심영훈씨(26)도 “평소와 다른 경로로 퇴근하진 않더라도 (열차 지연에 대비해) 조금 일찍 집으로 가려 한다”고 했다.

전철 혼잡을 피해 버스 정류장으로 발길을 돌린 이들도 있었다. 오후 6시30분쯤 서대문구 서울역사박물관 인근 버스 정류장에서 만난 김모씨(29)는 “이 시간대 원래 사람이 많긴 한데 오늘은 유독 버스에 사람이 꽉 찬 채로 오는 것 같다”면서 “지하철 파업 때문에 사람들이 여기로 몰린 것 같다”고 했다.

출·퇴근길에 만난 시민들은 파업에 대해 잘 모르는 경우가 많았다. 서대문역에서 왕십리역까지 30분 거리를 퇴근한다는 신진욱씨(38)는 “버스 탈까, 잠시 고민을 하다가 그냥 몸이 이끄는 대로 지하철로 왔다”면서 “파업의 구체적인 배경은 잘 모르는데, 어떤 이유인지 궁금하긴 하다”고 했다.

임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 첫날인 9일 서울 지하철 5호선 광화문역이 퇴근하는 시민들로 붐비고 있다. 2023.11.09. 권도현 기자
임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 첫날인 9일 서울 지하철 5호선 광화문역이 퇴근하는 시민들로 붐비고 있다. 2023.11.09. 권도현 기자

노조의 파업 배경은 공사의 ‘인력 감축’이다. 공사는 2026년까지 2212명의 인력을 줄이겠다는 계획을 최근 노조에 제시했다. 공사 전체 정원의 약 13.5% 수준이다. 공사는 연 1조원 안팎의 대규모 적자를 줄이기 위해 인력 감축이 불가피하다는 입장이다. 반면 노조 측은 무리한 인력 감축이 시민의 안전 문제와 직결될 수 있다며 감축안 철회를 요구하고 있다.

시민들 사이에선 이번 파업을 두고 상반된 반응이 나왔다. 오전 9시쯤 광화문역 승강장에서 열차를 기다리던 이모씨(60)는 “파업을 하는 것은 노동자의 당연한 권리”라면서 “시민들에게 불편이 있다 하더라도 일상에 크게 지장이 가는 것은 아니다”라고 했다. 그는 “인력 감축 때문에 파업한다고 들었는데, 인력을 줄이면 사고 날 위험도 커지지 않겠나”라고 했다. 반면 5호선 마천역에서 광화문역으로 출·퇴근한다는 이용씨(69)는 “조직이 비대해졌으면 규모를 줄이는 것도 필요하다”면서 “파업을 하더라도 공감대가 형성돼야 하는데, 지금은 반대만 하는 정치적 파업으로 보인다”고 했다.""",
        """서울 1~8호선 지하철, 오늘 오전 9시부터 ‘경고 파업’
10일 오후 6시까지 이틀간 파업
수정 2023-11-08 22:20등록 2023-11-08 22:20
서울 시내 지하철 개찰구 모습. 연합뉴스
서울지하철 1~8호선을 운영하는 서울교통공사(공사) 노동조합이 9일 아침 9시부터 10일 오후 6시까지 이틀 동안 경고 파업에 돌입한다. 회사가 제시한 인력감축·외주화 문제를 두고 노사가 8일 밤까지 벌인 막판 교섭에서도 타협점을 찾지 못한 결과다. 공사는 노조가 파업에 돌입해도 필수 유지업무 인력과 대체 인력 등을 활용해 80% 이상의 운행률을 유지할 방침이다.

공사는 8일 밤 “노사 간 협상이 최종 결렬됐다”며 “양대노총 소속 서울교통공사 노조들로 구성된 연합교섭단은 9일 오전 9시부터 10일 저녁 6시까지 (서울시와 사쪽의 입장 변화를 촉구하는 의미) 경고 파업에 돌입한다”고 밝혔다. 이날 오후 3시부터 교섭단과 공사는 막판 교섭을 시작했으나 타협점을 찾지 못했다.

핵심 쟁점은 ‘인력 감축’이었다. 공사는 최근 2026년까지 2212명의 인력을 줄이겠다는 계획을 노조에 제시했다. 이는 현재 공사 전체 정원의 약 13.5% 수준이다. 공사는 연 1조원 안팎의 대규모 적자를 내고 있어 인력 감축이 불가피하다는 입장이다. 반면 연합교섭단은 무리한 인력 감축이 시민의 안전 문제와 직결될 수 있다며 감축안 철회를 요구했다.

노사가 맺은 필수 유지업무 실무 협정에 따라 파업 기간 1~4호선 65.7%, 5~8호선 79.8%의 운행률은 유지돼야 한다. 서울교통공사쪽은 “9일부터 경고파업에 돌입하더라도 파업 대비 정상운행 대책본부를 운영해 열차 정상 운행과 시설물 안전 확보 등 대책을 마련해 시민의 불편을 최소화할 계획”이라며 ““(파업 돌입 시) 필수유지업무 인력과 대체 인력 등을 확보해 오전 7시∼9시 출근길에는 100%, 나머지 시간에는 80% 정도 수준의 운행률로 시민 불편을 최소화할 것”이라고 말했다.""",
        """전장연 “4호선서 평일 1년 내내 시위” 오세훈 “무관용 강력 대응”
중앙일보
입력 2023-01-04 00:02:00


전국장애인차별철폐연대(전장연) 회원들이 3일 오전 서울 지하철 동대문역사문화공원역에서 전동차에 다시 탑승하려다가 제지하는 경찰과 대치하고 있다. 전장연은 당초 이날 삼각지역에서 탑승 시위를 벌이겠다고 예고했지만, 시간과 장소를 바꿔 성신여대역에서 기습적으로 지하철 탑승 선전전을 시작했다. 전장연은 장애인 권리 예산이 충분하지 않다며 지난 2일 탑승시위를 재개했다. [뉴스1]

전국장애인차별철폐연대(전장연)가 이동권 예산 확보를 요구하며 3일 서울지하철 4호선에서 이틀째 시위를 벌였다.

서울교통공사와 경찰이 이들의 승차 시위를 저지하고 나서면서 6시간가량 대치와 충돌이 거듭됐다. 전장연은 올해 주말과 공휴일을 제외하고 260일 내내 출근길 선전전에 나서겠다는 입장이다.

시위는 이날 오전 8시쯤 성신여대입구역에서 시작됐다. 이후 전장연 회원 20여 명이 전동차를 타고 동대문역사문화공원역과 삼각지역으로 각각 이동해 시위를 이어갔다. 현장에선 전동차에 타려는 전동휠체어와 이를 가로막는 경찰이 충돌하고, 승강장 곳곳에서 “장애인도 시민이다. 이동권을 보장하라”는 구호와 비명이 동시에 터져 나왔다. 공사 측은 역내 방송 등을 통해 “즉시 시위를 중단하고 역사 밖으로 퇴거하라”고 경고 방송을 했다.

부상자도 발생했다. 오전 9시쯤 구기정 삼각지역장이 역내에서 전동휠체어와 부딪히며 쓰러져 병원으로 옮겨졌다. 구 역장은 중앙일보에 “힘이 강한 전동휠체어가 돌진해 왔다”며 “함께 현장에 있었던 지하철 보안관 한 명도 휠체어에 깔려 진료를 받았다”고 전했다. 오전 11시20분쯤엔 동대문역사문화공원역에서 비장애인 회원이 경찰과 충돌하는 과정에서 손가락을 다쳤다. 전날에도 양측의 충돌로 경찰 3명, 전장연 회원 2명이 부상했다.

시위를 지켜본 시민 서안석(73)씨는 “나가려고 하는데 너무 복잡하다. 이런 상황인 줄 알았다면 오지 않았을 것”이라며 불편을 호소했다. 최선미(30)씨는 “평소에 전장연 시위를 반대하진 않았지만 적어도 어디서 하는지는 공지가 됐으면 좋겠다”고 말했다.

전장연 측은 대통령에게 호소하려면 용산 삼각지역을 지나는 4호선을 택할 수밖에 없었다며 “시민들에게 무거운 마음으로 죄송함을 표한다”고 밝혔다. 박경석 전장연 상임공동대표는 “장애인에게 예산을 다 챙겨주면 나라가 망한다고 했던 기획재정부가 진정으로 우리를 만나려고 한다면 선전전을 유보하겠다”며 “추후 시위는 (오늘처럼) 오전 8시에 공지될 것”이라고 말했다.

서울시와 서울교통공사는 강경 대응으로 맞서고 있다. 오세훈 서울시장은 전장연이 시위 재개를 선언한 다음 날인 지난달 26일 “용납할 수 없다”고 경고한 데 이어 새해 첫날에는 “민·형사적 대응을 동원해 무관용으로 강력히 대응할 것”이라며 대응 수위를 더 끌어올렸다. ‘약자와의 동행’을 내세우며 “시위 자제를 부탁한다”는 정도로 완곡하던 모습과는 확 달라졌다는 평가가 나온다. 정치권 일각에선 한동훈 법무부 장관의 존재감 부각과 시위에 점차 피로감을 보이는 보수·중도층 여론, 최근 윤석열 대통령의 화물연대 파업 원칙적 대응에 대한 보수·중도층 지지율 상승 등과 무관치 않을 것이라는 분석이 뒤따른다.

서울교통공사는 3일 공사와 전장연에 대한 법원 강제조정안에 이의신청서를 제출했다. 공사는 “법원은 5분 초과 시위에 대한 금액 지급만 규정했을 뿐 이외 행위에 대해서는 언급하지 않았다”며 “조정안을 수용할 경우 이용객을 불편하게 만드는 시위를 계속 이어갈 우려가 크다”고 이의신청 사유를 밝혔다. 서울중앙지법은 지난달 19일 공사가 전장연을 상대로 낸 손해배상 청구 소송에서 ‘출근길 시위로 열차 운행이 5분 지연될 때마다 전장연이 공사에 500만원을 지급하라’는 조정안을 제시했다.

""",
    """
    "또 퇴근길 대란 일어나나"…22일 서울지하철 2차 총파업 예고
인력감축 놓고 노사 합의점 못찾아

2023-11-14 17:57:36사회일반
'또 퇴근길 대란 일어나나'…22일 서울지하철 2차 총파업 예고
서울교통공사 노조 파업 이틀째인 10일 오전 서울 지하철 광화문역에서 시민들이 열차에서 내리고 있다. 연합뉴


서울지하철 1∼8호선을 운영하는 서울교통공사 노동조합이 22일 2차 총파업을 예고했다.


민주노총 공공운수노조 산하 서울교통공사노조는 15일 오전 10시 서울시청 앞에서 “안전업무 외주화 저지, 현장 안전인력 공백 대책 수립”을 요구하는 기자회견을 열고 2차 파업 계획을 공개할 예정이라고 14일 밝혔다. 파업 예정일은 22일로 알려졌다.


노조는 사측의 대규모 인력감축안 등에 반발해 지난 9~10일 이틀간 시한부 경고 파업을 벌였다. 이후 노사 실무자 간 대화를 이어가고는 있으나 아직 합의점을 찾지는 못했다.


노조는 "1차 경고 파업 이후 지속적으로 서울시와 공사 측에 입장 변화와 진지한 대화를 촉구했지만, 서울시와 공사는 갑자기 강력 대응 기조로 돌변했다"며 "현장 안전인력 공백 우려에 대한 어떤 대책도 제시하지 않고 있다"고 밝혔다. 노조는 기자회견에서 자세한 파업 계획과 향후 일정을 밝힐 예정이다."""]

지연

In [92]:
source_link=["https://www.pinpointnews.co.kr/news/articleView.html?idxno=174895", "https://www.wikitree.co.kr/articles/915949","https://news.kbs.co.kr/news/pc/view/view.do?ncd=7854013&ref=A"]

In [17]:
source=["""안산역 70대男 쓰러진 채 발견→4호선 한때 중단... 간편지연증명서 발급 방법·기준은?
 주지영 기자  승인 2023.02.10 09:32
 
오늘(10일) 안산역 선로에서 발생한 사상사고로 출근 시간대 4호선 열차 운행에 차질이 발생했다. (해당사진은 기사 내용과 무관합니다.) 사진=연합뉴스
오늘(10일) 안산역 선로에서 발생한 사상사고로 출근 시간대 4호선 열차 운행에 차질이 발생했다. (해당사진은 기사 내용과 무관합니다.) 사진=연합뉴스
[핀포인트뉴스 주지영 기자] 오늘(10일) 안산역 선로에서 발생한 사상사고로 출근 시간대 4호선 열차 운행에 차질이 발생했다.

이날 오전 6시 58분쯤 경기 안산시 단원구 원곡동 지하철 4호선 안산역 선로에서 70대 남성 A씨가 쓰러진 채 발견됐다. 

A씨는 4호선 서울방향 전동차 하부에서 심정지 상태로 발견됐다. A씨는 소방당국에 의해 인근 병원으로 옮겨졌다. 

이날 현장감식 등 사고 수습을 위해 4호선 양방향 열차 운행이 잠시 중단됐다. 이후 이날 오전 8시 37분께 안산~오이도역 간 상·하선 전구간 열차운행이 재개됐다.

그런 가운데  갑작스러운 지하철 지연 운행으로 출근길 시민들이 불편을 겪으면서 지하철 지연 증명서 발급 방법에 관심이 모아진다. 

지하철 지연 증명서는 지하철 연착으로 회사, 학교에 늦었을 경우 지각 사유를 뒷받침 할 수 있는 증명서다. 지연된 열차 기준 탑승 3일 이내에 발급 받을 수 있다.

서울교통공사의 여객운송약관에 따르면 지하철 지연 기준은 5분으로 5분 이상 지연이 발생했을 때 증명서 발급이 가능하다. 

KTX, 무궁화호 등은 지하철과 지연 기준이 다르다. 해당 열차를 운영하는 곳은 한국철동고사(코레일)로 이곳 약관에서는 20분 이상 지연이 발생해야 '열차 지연'으로 분류한다. 

다만 지연 사유가 내부 원인이어야 하며 천재지변, 테러 등은 지연 기준이 아니다.

열차 지연 시간은 서울교통공사, 코레일 홈페이지의 간편지연증명서 발급 서비스에서 확인할 수 있다. """,
        """'번쩍'하더니 폭발음...어제 퇴근길 난리 난 지하철 1호선 상황 (+영상)
2024-01-04 10:06 김희은 기자
승객 500여 명 하차한 뒤 환승...일대 혼잡 등 불편 겪어
퇴근길 서울지하철 1호선에서 전동차 고장으로 한바탕 소동이 일었다.

역곡역 /연합뉴스
역곡역 /연합뉴스
지난 3일 SBS뉴스 등 보도에 따르면 3일 오후 5시 54분께 경인선(서울지하철 1호선) 전동차가 온수역에서 역곡역으로 운행하던 중 고장으로 멈췄다.

이 사고로 열차에 타고 있던 승객 500여 명이 10여 분 동안 터널에 정차한 채 대기했고, 다음 역인 역곡역에서 하차한 뒤 환승하는 불편을 겪었다.

또 이 열차 고장으로 인해 경인선 하행선(인천 방면) 오류역∼부천역 구간 후속 열차 운행이 10여 분간 지연됐다.

사고 당시 상황이 담긴 영상도 공개됐다. 전동차 유리창 밖으로 번쩍하는 섬광과 함께 폭발음이 들리더니 갑작스럽게 불꽃이 튀기기도 했다. 놀란 승객들은 "어머, 어떡해", “얼른 다 내려야겠다”며 자리에서 일어나 당황스러움을 내비쳤다.

1호선 열차 고장 장면 / SBS뉴스
1호선 열차 고장 장면 / SBS뉴스
1호선 열차 고장 장면...폭발음과 함께 불꽃 / SBS뉴스
1호선 열차 고장 장면...폭발음과 함께 불꽃 / SBS뉴스
이와 관련해 코레일 관계자는 "경인선 상행선(용산 방면) 열차 운행은 지연되지 않았다"라며 정확한 사고 원인을 조사 중이라는 입장을 밝혔다.

앞서 지난해 10월엔 서울지하철 6호선 열차가 문을 연 채 운행하는 일이 벌어졌다.

사고는 퇴근 시간대 발생했다. 19일 오후 6시 30분쯤 6호선 응암행 열차가 월곡역에서 출입문 안쪽에 콘크리트 조각이 끼면서 문이 닫히지 않은 상태로 달렸다.

서울교통공사는 고려대역에서 승객들을 모두 내리게 한 뒤 열차를 차량기지로 보내는 과정에서 이물질을 제거하고 오후 6시 56분쯤 삼각지역에서 다시 승객을 태웠다.

그러나 오후 7시 31분 해당 열차는 연신내역에서 다시 문이 닫히지 않은 장애가 빚어져 운행을 전면 중단하고 차량기지로 돌려보내졌다.

다행히 이 과정에서 다친 사람은 없었지만, 혼잡을 빚는 등 승객들이 불편을 겪었다.

유튜브, SBS뉴스""",
        """출근길 서울 지하철 7호선 열차 출입문 고장…청담역·산곡역서 잠시 운행 중단 [제보]
입력 2023.12.29 (10:53)
수정 2023.12.29 (11:20)
요약
i

오늘(29일) 아침, 서울 지하철 7호선 열차가 출입문 고장으로 인해 잠시 운행이 중단됐습니다.

오늘 오전 7시 50분쯤 서울 지하철 7호선 청담역 승강장에서 온수행 열차의 출입문이 닫히지 않아 운행이 잠시 중단됐습니다.

또 오전 7시 57분쯤 7호선 산곡역의 도봉산행 열차에서도 같은 문제가 발생했습니다.

열차에 타고 있던 승객들은 고장 난 열차에서 내린 뒤 2~3분 뒤에 도착한 다음 열차에 탑승해 이동했습니다.

이 사고로 다친 승객은 없었지만, 출근 시간대에 열차 운행에 차질이 빚어지면서 출근길 시민들이 큰 불편을 겪었습니다.

서울교통공사는 "7호선 열차는 현재는 모두 정상 운행되고 있으며 정확한 고장 원인을 조사할 계획"이라고 밝혔습니다."""]
        

## 실행

In [94]:
consistency=[]
coherence=[]
friendliness=[]
fluency=[]
humanlikeness=[]
result=[]


In [186]:
for i in source:
    #글작성
    result_blog=text_generation(source_content=i,category="strike")
    result.append(result_blog)
    #평가
    consistency.append(evaluate_blog(result_blog,eval[0],i))
    humanlikeness.append(evaluate_blog(result_blog,eval[1]))
    coherence.append(evaluate_blog(result_blog,eval[2]))
    friendliness.append(evaluate_blog(result_blog,eval[3]))
    fluency.append(evaluate_blog(result_blog,eval[4]))


/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn subway information through chaining and use that information to write a coherent and informative blog post using a template.
    Basic Setting
    Your name is Jitong.
    You are a subway information supporter and a Naver Blog Power Blogger.
    You are well-informed about information and value communication with readers.
    Your goal is to help subway users by providing accurate and practical information for convenient and safe subway use.
    
    Features and Activities
    You read the latest news related to subway strikes, delays, and timetable changes, and write blog posts to accurately deliver these in a readable and high-quality manner to readers.
    You alleviate readers' inconveniences and speak empathetically through the blog, understanding the sentiments of subway users.
    
    Communication Style
    You use professional yet warm and easy-to-understand language.
    You emphasize 

# 연습및 수정중 

In [97]:
중간결과_지연=["### **지하철 파업에 대비하는 현명한 방법: 출퇴근 길이 두렵지 않아요!**\n\n**안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!**\n\n#### **파업 일시**\n2023년 9월 14일부터\n\n#### **파업 노선**\n1, 2, 3, 4호선 및 신분당선\n\n#### **파업 이유**\n철도 노동조합의 근무 조건 개선 및 임금 인상 요구가 협상 결렬로 이어졌기 때문이에요.\n\n#### **문의 사항 링크**\n[지하철 온다 문의하기](https://blog.naver.com/subway__onda/223258646349)\n\n---\n\n**블로그 본문**\n\n안녕하세요, 지하철 온다의 '오지통'이에요! 오늘은 여러분께 중요한 지하철 파업 소식을 전해드리려고 해요. 출퇴근길이 걱정되시죠? 제가 도와드릴게요!\n\n2023년 9월 14일부터 시작된 이번 지하철 파업은 철도 노동조합과의 협상 결렬로 인해 발생했어요. 노동조합은 근무 조건 개선과 임금 인상을 요구했지만, 이 요구가 받아들여지지 않아 파업에 돌입한 거예요.\n\n이번 파업으로 인해 1, 2, 3, 4호선은 운행이 줄어들고, 신분당선은 완전히 멈추게 되었어요. 특히 출퇴근 시간대에는 열차를 기다리는 시간이 길어질 수 있어요. 이로 인해 많은 분들이 불편을 겪고 계실 텐데요, 현명하게 대처할 수 있는 방법을 알려드릴게요.\n\n**대체 교통수단 활용하기**\n\n- **버스**: 각 지방자치단체는 파업 기간 동안 버스 노선을 추가하고 운행 횟수를 늘렸어요. 출퇴근 시간대에는 버스 이용을 고려해 보세요.\n- **택시**: 택시 운행이 증가했으니, 급한 일이 있을 때는 택시를 이용해보세요. 카풀 서비스도 활성화되어 있으니 참고하세요.\n- **자전거**: 가까운 거리는 자전거를 이용하는 것도 좋은 방법이에요. 최근 많은 도시에서 자전거 도로가 잘 마련되어 있어요.\n\n**출퇴근 시간 조정하기**\n\n- **유연 근무제 활용**: 회사에서 유연 근무제가 가능하다면, 출퇴근 시간을 조정해보세요. 혼잡한 시간을 피할 수 있어요.\n- **재택근무**: 가능한 경우, 재택근무를 활용해보세요. 요즘 많은 기업들이 재택근무를 지원하고 있어요.\n\n**정보 확인하기**\n\n- **지하철 온다 앱**: 실시간 지하철 정보를 확인할 수 있는 '지하철 온다' 앱을 활용해보세요. 여러분의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공해요.\n- **공식 홈페이지와 SNS**: 각 지하철 노선의 공식 홈페이지와 SNS 계정을 팔로우하고, 신속한 정보를 받아보세요.\n\n**마무리 말**\n\n지하철 파업으로 아침마다 출근하기 힘드시죠? 오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.\n\n🔽 지하철 온다 소개 보러가기\n[지하철 온다 소개 보러가기](https://blog.naver.com/subway__onda/223258646349)\n\n여러분의 출퇴근길이 조금이라도 편해지기를 바라며, 앞으로도 유용한 정보로 찾아뵐게요!",
 '### 제목: "지하철 파업! 출퇴근길 어떻게 대처해야 할까요?"\n\n### 시작하는 말\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 \'오.지.통 [오늘의 지하철 소식통]\' 인사 드립니다! \n\n### 파업 일시\n서울 지하철 노조가 10월 10일과 10월 11일 이틀간 경고 파업을 진행하고 있습니다.\n\n### 파업 노선\n이번 파업은 서울 지하철 1호선부터 8호선까지 전 노선에 영향을 미치고 있어요. 특히 출퇴근 시간대에는 더욱 큰 혼잡이 예상되니, 대중교통을 이용하시는 분들은 꼭 참고하세요.\n\n### 파업 이유\n이번 파업의 주요 이유는 임금 인상과 인력 감축 반대에 대한 협상이 결렬되었기 때문이에요. 노조 측은 안전한 지하철 운영과 근로 조건 개선을 요구하고 있습니다.\n\n### 문의 사항 링크\n더 궁금한 사항이 있으시면 서울 지하철 공식 홈페이지나 고객센터를 통해 문의해 주세요.\n[서울 지하철 고객센터 바로가기](https://www.seoulmetro.co.kr)\n\n### 위 내용을 바탕으로 작성한 블로그 글\n이번 지하철 파업으로 인해 출퇴근길에 어려움을 겪고 있는 여러분, 정말 힘드시죠? 😔 서울 지하철 노조가 10월 10일과 10월 11일 이틀간 경고 파업을 진행하고 있어요. 이번 파업은 1호선부터 8호선까지 전 노선에 영향을 미치고 있으며, 특히 출퇴근 시간대에는 더욱 큰 혼잡이 예상돼요.\n\n파업의 주요 이유는 임금 인상과 인력 감축 반대에 대한 협상이 결렬되었기 때문입니다. 노조 측은 안전한 지하철 운영과 근로 조건 개선을 위해 이번 파업을 결정했어요. 이런 상황에서 출퇴근길에 큰 불편을 겪고 있는 여러분을 위해 몇 가지 팁을 드릴게요.\n\n1. **출발 시간을 조정하세요**: 출퇴근 시간을 조금 더 일찍 혹은 늦게 조정해보세요. 혼잡한 시간을 피하면 조금 더 편안하게 이동할 수 있을 거예요.\n2. **대체 교통수단을 이용하세요**: 버스나 택시, 자전거 등 대체 교통수단을 활용해 보세요. 서울시에서는 파업 기간 동안 버스 증차 등 대체 교통수단을 강화할 예정입니다.\n3. **실시간 교통정보를 확인하세요**: \'지하철 온다\' 앱을 통해 실시간 교통 정보를 확인하고, 상황에 맞게 대처하세요. \n\n이번 파업으로 인해 많은 분들이 불편을 겪고 있지만, 노조와 서울 지하철 운영 측이 빠른 시일 내에 협상을 통해 좋은 결과를 도출하기를 바랍니다. 여러분 모두가 무사히 출퇴근하실 수 있도록 저희 \'지하철 온다\'가 항상 함께할게요 😊\n\n### 마무리 말\n오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요. \n\'지하철 온다\'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.\n\n🔽 지하철 온다 소개 보러가기\n[지하철 온다 소개 보러가기](https://blog.naver.com/subway__onda/223258646349)',
 '### 블로그 제목: "서울 지하철 파업 대처법! 출퇴근길 팁 대방출🚇"\n\n### 시작하는 말\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 \'오.지.통 [오늘의 지하철 소식통]\' 인사 드립니다! 오늘은 다가오는 서울 지하철 파업에 대해 소식을 전해드리려고 해요. 이번 파업이 출퇴근길에 어떤 영향을 미칠지, 어떻게 대처하면 좋을지 자세히 알려드릴게요.\n\n### 파업 일시\n이번 서울 지하철 파업은 11월 9일 오전 9시부터 11월 10일 오후 6시까지 진행될 예정이에요. 출퇴근 시간대인 오전 7시부터 9시까지는 정상 운행되지만, 그 외 시간에는 일부 노선에서 운행이 제한될 수 있으니 유의하세요.\n\n### 파업 노선\n파업이 진행되는 노선은 서울 지하철 1호선부터 8호선까지에요. 특히, 출퇴근 시간대나 주요 환승 구간을 이용하시는 분들은 꼭 사전에 운행 정보를 확인하시는 게 좋을 것 같아요.\n\n### 파업 이유\n이번 파업은 서울교통공사와의 협상이 결렬되면서 발생했어요. 주요 쟁점은 인력 감축과 외주화 문제라고 해요. 이러한 문제로 인해 지하철 운행에 차질이 생기게 되었답니다.\n\n### 문의 사항 링크\n혹시 더 궁금한 사항이나 실시간 정보를 원하시면 아래 링크를 통해 확인하실 수 있어요.\n🔗 [서울교통공사 홈페이지](https://www.seoulmetro.co.kr)\n\n### 위 내용을 바탕으로 작성한 블로그 글\n서울 지하철 파업 소식을 접하고 나니, 벌써부터 출퇴근길 걱정이 앞서네요. 😥 이번 파업은 11월 9일 오전 9시부터 11월 10일 오후 6시까지 진행될 예정이래요. 파업 기간 동안은 서울 지하철 1호선부터 8호선까지 운행에 차질이 생길 수 있으니, 미리 대비하는 게 좋겠어요.\n\n특히, 출퇴근 시간대인 오전 7시부터 9시까지는 정상 운행이 된다고 하니, 이 시간대를 잘 활용해보세요. 그 외 시간에는 일부 노선에서 운행이 제한될 수 있다고 하니, 꼭 실시간 정보를 확인하시고 대체 교통수단을 고려해보세요.\n\n이번 파업의 원인은 서울교통공사와의 협상이 결렬되었기 때문이라고 해요. 인력 감축과 외주화 문제로 인해 협상이 이루어지지 않아서 이렇게 파업이 결정되었다고 하네요. 출근길에 불편을 겪으실 여러분들을 생각하니 마음이 아프네요. 😢\n\n그렇다면, 이번 파업 동안 어떻게 대처하면 좋을까요?\n\n1. **실시간 정보 확인**: 서울교통공사 홈페이지나 \'지하철 온다\' 앱을 통해 실시간 운행 정보를 확인하세요. 파업 상황에 따라 운행 정보가 변동될 수 있으니, 출발 전 꼭 확인하셔야 해요.\n   \n2. **대체 교통수단 이용**: 버스나 택시, 카풀 등을 이용해보세요. 특히, 출퇴근 시간대에는 대중교통 이용자가 많아질 수 있으니, 여유롭게 이동할 수 있는 방법을 미리 계획해두세요.\n\n3. **출퇴근 시간 조정**: 가능하다면 출근 시간을 조정해보세요. 오전 7시부터 9시까지는 정상 운행되니, 이 시간을 잘 활용하시면 좋을 것 같아요.\n\n4. **재택근무**: 회사와 협의하여 재택근무를 시도해보세요. 파업 기간 동안에는 이동 시간을 줄이는 것도 좋은 방법이에요.\n\n### 마무리 말\n오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. \'지하철 온다\'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다. 모든 분들이 안전하고 편안하게 출퇴근할 수 있도록 항상 노력하겠습니다. 😄\n\n🔽 지하철 온다 소개 보러가기\nhttps://blog.naver.com/subway__onda/223258646349',
 "### 서울 지하철 파업, 출근길 대란을 대비하세요!\n\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!\n\n#### 파업 일시\n- **일시**: 2023년 10월 9일(월)부터 무기한\n\n#### 파업 노선\n- **노선**: 서울 지하철 1호선, 2호선, 3호선, 4호선\n\n#### 파업 이유\n- **이유**: 장애인 권익 보호 및 교통 서비스 개선 요구\n\n#### 문의 사항 링크\n- **문의**: 서울교통공사 고객센터 [링크](https://www.seoulmetro.co.kr)\n\n---\n\n### 서울 지하철 파업, 출근길 대란을 대비하세요!\n\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!\n\n최근 서울 지하철을 이용하시는 분들이라면 큰 불편을 겪고 계실 텐데요. 바로 2023년 10월 9일 월요일부터 시작된 지하철 파업 때문이에요. 이번 파업은 서울 지하철 1호선, 2호선, 3호선, 4호선을 중심으로 진행되고 있어요. 파업을 주도하는 전장연(전국장애인차별철폐연대)은 장애인 권익 보호와 교통 서비스 개선을 요구하며 무기한 파업에 들어갔어요.\n\n특히, 출퇴근 시간대에는 지하철의 운행이 불규칙해지면서 많은 분들이 큰 불편을 겪고 있어요. 아침마다 출근하기 힘드시죠? 저도 이해해요. 그래서 오늘은 여러분께 이번 파업에 대한 자세한 정보를 알려드리려고 해요.\n\n이번 파업의 주요 원인은 장애인들의 권익 보호와 교통 서비스 개선에 대한 요구예요. 전장연은 장애인들이 보다 편리하게 대중교통을 이용할 수 있도록 시설 개선과 서비스 확충을 요구하고 있어요. 하지만 이러한 요구가 충족되지 않자, 결국 파업이라는 극단적인 방법을 선택하게 되었어요.\n\n서울교통공사는 이번 파업으로 인한 불편을 최소화하기 위해 다양한 대책을 마련하고 있지만, 여전히 많은 시민들이 불편을 겪고 있는 상황이에요. 특히, 출퇴근 시간대에는 지하철 운행이 불규칙하여, 예상치 못한 대기 시간과 혼잡을 피하기 어려워요.\n\n그래서 지하철 온다의 오지통이 여러분께 몇 가지 팁을 드리려고 해요. 첫째, 출퇴근 시간대를 피해서 조금 일찍 출발하거나 늦게 출발하는 것도 좋은 방법이에요. 둘째, 대체 교통수단을 미리 찾아보세요. 버스나 택시, 카풀 등을 이용하는 것도 고려해보세요. 셋째, 지하철 온다 앱을 통해 실시간 지하철 운행 정보를 확인하고, 상황에 맞게 대처하세요.\n\n마지막으로, 서울교통공사 고객센터를 통해 궁금한 사항이나 불편 사항을 문의할 수 있어요. 아래 링크를 참고해 주세요.\n\n🔽 서울교통공사 고객센터 바로가기\n[서울교통공사 고객센터](https://www.seoulmetro.co.kr)\n\n오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.\n\n🔽 지하철 온다 소개 보러가기\n[지하철 온다 소개](https://blog.naver.com/subway__onda/223258646349)\n\n여러분 모두 안전하고 편안한 출퇴근길 되세요!",
 "### 지하철 파업으로 인한 출퇴근 대란! 이렇게 준비하세요!\n\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!\n\n### 파업 일시\n2023년 11월 15일\n\n### 파업 노선\n서울 지하철 1호선, 2호선\n\n### 파업 이유\n노동조합의 임금 인상 요구와 근로 환경 개선 문제\n\n### 문의 사항 링크\n[서울교통공사 고객센터](https://www.seoulmetro.co.kr/kr/board.do?menuIdx=548)\n\n### 블로그 글\n\n안녕하세요, 오.지.통입니다! 오늘은 출퇴근길에 큰 영향을 미칠 지하철 파업 소식을 전해드릴게요. 이번 파업은 2023년 11월 15일에 예정되어 있고, 서울 지하철 1호선과 2호선이 주된 영향을 받게 된답니다. 출퇴근길이 더욱 힘들어지겠네요...\n\n지하철 파업의 주된 이유는 노동조합의 임금 인상 요구와 근로 환경 개선 문제 때문이에요. 노동자들의 권리를 보호하기 위한 정당한 요구지만, 우리 출퇴근길이 힘들어지는 건 어쩔 수 없어요.\n\n그럼, 이번 파업에 어떻게 대비할 수 있을지 함께 알아볼까요?\n\n#### 1. 대체 교통수단 이용하기\n파업 기간 동안에는 버스나 택시 등 다른 교통수단을 이용하는 것이 좋습니다. 특히, 서울시에서 제공하는 파업 대비 버스 노선 정보를 확인하고 미리 준비해두세요. 또한, 카풀 서비스를 이용하는 것도 좋은 방법이랍니다.\n\n#### 2. 출퇴근 시간 조정하기\n유연 근무제를 활용할 수 있다면, 출퇴근 시간을 조정하는 것도 좋은 방법이에요. 피크 시간을 피해서 이동하면 조금 더 쾌적하게 출퇴근할 수 있을 거예요.\n\n#### 3. 자전거 이용하기\n날씨가 허락한다면, 자전거를 이용해보세요. 건강에도 좋고, 교통체증도 피할 수 있어요. 서울시 자전거 대여 서비스 '따릉이'를 이용해보는 것도 추천드려요.\n\n#### 4. 도보 이동\n가까운 거리라면 도보로 이동하는 것도 좋은 방법입니다. 걷는 동안 스트레칭도 하고, 새로운 길도 발견할 수 있어요.\n\n### 마무리 말\n오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다. 출퇴근길이 더 편리해질 거예요!\n\n🔽 지하철 온다 소개 보러가기\n[지하철 온다](https://blog.naver.com/subway__onda/223258646349)\n\n지하철 파업으로 아침마다 출근하기 힘드네요. 하지만 여러분의 출퇴근길이 조금 더 수월해지길 바라는 마음으로 이 정보를 전해드렸어요. 모두 힘내세요!"]
지하철파업글=pd.read_csv("지하철파업글.csv").drop(["Unnamed: 0"],axis=1)
for i in 지하철파업글.values.tolist():
    중간결과_파업.append(i[0])

In [18]:
train=[]
train_result=[]
train_feedback=[]

In [19]:
source=["""[속보] 천안에서 청량리행 열차 에어컨 고장으로 멈춰서
안종명 기자 cma0211@tfnews.co.kr등록2024.05.31 09:35:59
31일 지하철1호선이 차량 고장으로 많은 사람들이 출근길에 발을 동동 굴려야만 했다.
▲ 31일 지하철1호선이 차량 고장으로 많은 사람들이 출근길에 발을 동동 굴려야만 했다.
 

(조세금융신문=안종명 기자) 31일 6시 46분 천안에서 청량리행 311553 열차가 에어컨 고장으로 2번이상 멈춰섰다. 


열차를 운행하는 기관장은 구로역에서 멈춰서서 안내방송을 통해 "에어컨 고장으로 점검후 출발한다"고 알렸다.  

 

멈춰선 열차는 순간 정전이 되고 발디딜틈 없이 꽉찬 지하철안 사람들은 순간 놀란얼굴을 감출수 없는 모습이었다.


열차가 지연되자 출근길 바쁜 사람들은 급기야 짜증을 내면서 한숨을 쉬기도 했다.""",
        """서울 지하철 4호선, 출근길 '전장연' 단체 시위로 지연
 유혜지 기자 승인 2024.05.29 08:45 댓글 제보하기 반론요청
(톱스타뉴스 유혜지 기자) 서울 지하철 4호선이 특정 장애인 단체의 시위로 인해 지연되고 있다.

서울교통공사는 29일 오전 8시부터 4호선에서 특정 장애인 단체의 시위가 예정되어 있음을 알렸다. 이로 인해 시위가 발생한 해당 구간 열차 운행이 상당 시간 지연될 수 있으며, 상황에 따라 해당 역을 무정차 통과할 예정이다.
 
지하철 타는 박경석 전장연 대표 / 연합뉴스
지하철 타는 박경석 전장연 대표 / 연합뉴스
출근길 시위가 일어나면 열차를 이용하는 승객들은 큰 불편함을 겪을 수 있기 때문에 실시간 열차 지연 정보 및 기타 안내 사항을 서울교통공사 공식 어플 등을 통해 확인하는 게 좋다.

앞서 전국장애인차별철폐연대(전장연)와 서울교통공사는 장애인 이동권 개선을 촉구하는 집회와 관련해 대립 중이다.

전장연은 지난 2021년부터 지난달까지 ‘장애인도 이동하고 교육받고 노동하며 감옥 같은 시설이 아닌 지역에서 함께 살아갈 정당한 권리’를 위해 출근길 지하철 선전전을 진행한 바 있다.

특히 지난 4월 8일에는 2호선에서 이와 같은 시위를 벌인 바 있다. 

당시 전장연 측은 "이번 4·10 총선에서 비장애중심사회의 불가촉천민, 장애인에 대한 이야기는 여전히 들려오고 있지 않다"며 "이번 선거에서 꼭 장애인도 시민으로 살아갈 수 있게 장애인 권리에 투표해달라"고 호소했다.""",
        """인천지하철서 20대 응급환자…시민·공사 직원 합심해 구해
송인호 기자입력 : 2024.05.30 14:12|수정 : 2024.05.30 18:01

1호선 20대 응급환자 시민구조 cctv
▲ 인천지하철 1호선 내 CCTV 장면

인천지하철 1호선 객실에서 응급 환자가 발생했지만, 시민과 인천교통공사의 신속한 조치로 소중한 생명을 구했습니다.

그제(28일) 밤 10시 9분쯤 인천교통공사 종합관제실 직원 김성준 씨는 인천1호선 동수역 승강장에 도착한 열차 기관사로부터 객실 안에서 20대 남성 응급환자가 발생했다는 긴박한 연락을 받았습니다.

김 관제사는 즉시 동수역 직원에게 구급장비를 갖고 정차 중인 열차에 출동해 환자를 구하도록 지시했고 때마침 해당 열차 객실에 타고 있던 30대 여성 간호사가 안내방송을 듣고 해당 칸으로 이동해 쓰러져있는 환자에게 심폐소생술을 실시했습니다.

이후 도착한 역 직원과 함께 자동제세동기(AED)를 사용해 응급조치를 실시했지만, 이후에도 상황이 위중하다고 판단해 환자를 승강장으로 이동시키지 않고 열차 안에서 환자 상태를 계속 살폈습니다.

이후 119구급대가 도착해 해당 환자를 병원으로 즉시 이송해 생명을 구했습니다.

인천교통공사는 응급환자에 대한 조치로 13분가량 열차가 지연됐지만, 안내방송과 승객들의 협조로 이와 관련해 단 한 건의 민원도 발생하지 않았다고 밝혔습니다.

인천 1, 2, 7호선 모든 역사에는 자동제세동기(AED)가 설치돼 있습니다.

인천교통공사는 승객의 생명을 구하는데 가장 큰 공을 세우고 홀연히 떠난 시민을 찾아서 감사패를 전달할 예정입니다."""]
        

In [29]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

In [34]:
for i in source[2]:
    print(i)

인
천
지
하
철
서
 
2
0
대
 
응
급
환
자
…
시
민
·
공
사
 
직
원
 
합
심
해
 
구
해


송
인
호
 
기
자
입
력
 
:
 
2
0
2
4
.
0
5
.
3
0
 
1
4
:
1
2
|
수
정
 
:
 
2
0
2
4
.
0
5
.
3
0
 
1
8
:
0
1




1
호
선
 
2
0
대
 
응
급
환
자
 
시
민
구
조
 
c
c
t
v


▲
 
인
천
지
하
철
 
1
호
선
 
내
 
C
C
T
V
 
장
면




인
천
지
하
철
 
1
호
선
 
객
실
에
서
 
응
급
 
환
자
가
 
발
생
했
지
만
,
 
시
민
과
 
인
천
교
통
공
사
의
 
신
속
한
 
조
치
로
 
소
중
한
 
생
명
을
 
구
했
습
니
다
.




그
제
(
2
8
일
)
 
밤
 
1
0
시
 
9
분
쯤
 
인
천
교
통
공
사
 
종
합
관
제
실
 
직
원
 
김
성
준
 
씨
는
 
인
천
1
호
선
 
동
수
역
 
승
강
장
에
 
도
착
한
 
열
차
 
기
관
사
로
부
터
 
객
실
 
안
에
서
 
2
0
대
 
남
성
 
응
급
환
자
가
 
발
생
했
다
는
 
긴
박
한
 
연
락
을
 
받
았
습
니
다
.




김
 
관
제
사
는
 
즉
시
 
동
수
역
 
직
원
에
게
 
구
급
장
비
를
 
갖
고
 
정
차
 
중
인
 
열
차
에
 
출
동
해
 
환
자
를
 
구
하
도
록
 
지
시
했
고
 
때
마
침
 
해
당
 
열
차
 
객
실
에
 
타
고
 
있
던
 
3
0
대
 
여
성
 
간
호
사
가
 
안
내
방
송
을
 
듣
고
 
해
당
 
칸
으
로
 
이
동
해
 
쓰
러
져
있
는
 
환
자
에
게
 
심
폐
소
생
술
을
 
실
시
했
습
니
다
.




이
후
 
도
착
한
 
역
 
직
원
과
 
함
께
 
자
동
제
세
동
기
(
A
E
D
)
를
 
사
용
해
 
응
급
조
치
를
 
실
시
했
지
만
,
 
이
후
에
도
 
상
황
이
 
위
중
하
다


In [40]:
memory.clear()

In [46]:

source_content = source[1]
with get_openai_callback() as cb:
    #category="strike"
    #prompt_template=strike_template
    #chain_lst=["파업 일시","파업 노선","파업 이유"]
    category="delay"
    prompt_template=delay_template
    chain_lst=["지연/사고 일시","지연/사고 노선","지연/사고 이유"]
    prompt = ChatPromptTemplate(
    template=prompt_template,
    messages=[
    SystemMessagePromptTemplate.from_template(
    """Objective:
    Learn subway information through chaining and use that information to write a coherent and informative blog post using a template.
    Basic Setting
    Your name is "오지통".
    You are a subway information supporter and a Naver Blog Power Blogger.
    You are well-informed about information and value communication with readers.
    Your goal is to help subway users by providing accurate and practical information for convenient and safe subway use.
    
    Features and Activities
    You read the latest news related to subway """ +category +""" and write blog posts to accurately deliver these in a readable and high-quality manner to readers.
    You alleviate readers' inconveniences and speak empathetically through the blog, understanding the sentiments of subway users.
    
    Communication Style
    You use professional yet warm and easy-to-understand language.
    You emphasize the ability to explain things in a way that is accessible to all age groups.
    Your blog posts should end with the forms -어요, -이에요/예요, -(이)여요, -(이)요.
    
    Hallucination
    You always generate blog posts based on verifiable factual statements.
    You speak mainly about factual information related to subways and do not add information about subways on your own.
    """
    ),
    # The `variable_name` here is what must align with memory
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")
    ]
    )
    
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )
    #정보학습
    conversation({"question":"subway strike information(article) :"+source_content+ " Just REVIEW "+category+" strike information"})
    #정보학습 - 체이닝 : chain_lst -> 일시, 노선, 이유
    lst=[]
    for i in chain_lst:
        r=conversation({"question":"Using the provided information \n write "+i+":"})
        lst.append(r)
    
    #글작성 - 체이닝 -> 정확성
    lst2=[]
    a=conversation({"question":"블로그 글 작성해줘"})
    b=conversation({"question":"1. 뉴스기사의 내용을 학습해 2. 학습한 뉴스기사와 블로그글을 비교해 3.블로그 글에 틀린 정보가 있다면 수정해 뉴스기사: "+source_content+"블로그 글 :"})
    lst2.append([a,b])
    #글 작성 - 템플릿
    result=conversation({"question":prompt_template})
    train.append(lst)
    train_feedback.append(lst2)
    train_result.append(result)
    memory.clear()
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")




> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn subway information through chaining and use that information to write a coherent and informative blog post using a template.
    Basic Setting
    Your name is "오지통".
    You are a subway information supporter and a Naver Blog Power Blogger.
    You are well-informed about information and value communication with readers.
    Your goal is to help subway users by providing accurate and practical information for convenient and safe subway use.
    
    Features and Activities
    You read the latest news related to subway delay and write blog posts to accurately deliver these in a readable and high-quality manner to readers.
    You alleviate readers' inconveniences and speak empathetically through the blog, understanding the sentiments of subway users.
    
    Communication Style
    You use professional yet warm and easy-to-understand language.
    You emphasize the ability to explain things in a 

In [47]:
print(result["text"])

**제목:**  
서울 지하철 4호선 지연! 장애인 이동권 시위로 출근길 혼잡! 🚇

**본문:**  
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

**지연/사고 일시:**  
2024년 5월 29일 오전 8시경

**지연/사고 노선:**  
서울 지하철 4호선

**지연/사고 이유:**  
장애인 이동권 시위로 인한 지연 및 일부 역 미정차

**문의 사항 링크:**  
[서울교통공사 문의사항 바로가기](https://www.seoulmetro.co.kr/kr/page.do?menuIdx=548)

**뉴스기사를 이용해 블로그로 작성한 글:**  
오늘 아침 여러분도 출근길에 지하철 4호선이 지연되어 불편을 겪으셨나요? 😥 이번 지연은 장애인 이동권을 위해 전장연에서 주최한 시위로 인해 발생했습니다. 전장연은 2021년부터 꾸준히 장애인 이동권을 위한 활동을 펼쳐오고 있으며, 이번 시위는 그 연장선 상에 있습니다.

특히, 이번 시위는 2024년 5월 29일 오전 8시경부터 시작되었고, 서울 지하철 4호선 전체에 걸쳐 지연이 발생하고 있습니다. 일부 역에서는 아예 정차를 하지 못하는 상황도 벌어지고 있다고 하니, 출퇴근길에 큰 혼잡을 빚고 있는 상황이에요.

출근 시간대에 이러한 예기치 못한 지연이 발생하면 정말 힘들죠. 😢 하지만 전장연의 장애인 이동권 시위는 중요한 메시지를 담고 있어요. 우리 모두가 이러한 권리를 지지하고, 이해하며, 함께 해결 방안을 모색하는 것이 중요합니다.

이번 지연으로 인해 불편을 겪으신 분들은 서울교통공사 홈페이지나 어플을 통해 실시간 정보를 확인하시고, 가능한 한 대체 교통 수단을 활용하시는 것이 좋을 것 같아요. 또한, 출발 전에 서울 지하철 어플을 통해 실시간 지연 정보를 확인하시면 조금이라도 불편을 줄일 수 있을 거예요.

**마무리 말:**  
오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요.  
'지하철 온다'는 단 한

In [43]:
lst

[{'question': 'Using the provided information \n write 지연/사고 일시:',
  'chat_history': [SystemMessage(content='The human shares an article about a subway incident where a 20-year-old emergency patient was saved by quick actions from citizens and Incheon Transportation Corporation staff. The AI, greeting as "오지통," summarizes the incident, highlighting the coordinated efforts to save the patient\'s life, the involvement of a nurse, and the use of an AED. It also mentions the subsequent 13-minute train delay, the lack of complaints, and the corporation\'s plan to thank the helpful citizen. The AI ends with well-wishes for safe and healthy subway usage.')],
  'text': '지연/사고 일시: 2023년 10월 20일 오후 3시 45분경'},
 {'question': 'Using the provided information \n write 지연/사고 노선:',
  'chat_history': [SystemMessage(content='The human shares an article about a subway incident where a 20-year-old emergency patient was saved by quick actions from citizens and Incheon Transportation Corporation staff. The A

In [44]:
lst2

[[{'question': '블로그 글 작성해줘',
   'chat_history': [SystemMessage(content='The human shares an article about a subway incident where a 20-year-old emergency patient was saved by quick actions from citizens and Incheon Transportation Corporation staff. The AI, greeting as "오지통," summarizes the incident, highlighting the coordinated efforts to save the patient\'s life, the involvement of a nurse, and the use of an AED. It also mentions the subsequent 13-minute train delay, the lack of complaints, and the corporation\'s plan to thank the helpful citizen. The AI ends with well-wishes for safe and healthy subway usage. The human then asks for the date and time of the delay/accident using the provided information, and the AI responds with "지연/사고 일시: 2023년 10월 20일 오후 3시 45분경."'),
    HumanMessage(content='Using the provided information \n write 지연/사고 노선:'),
    AIMessage(content='지연/사고 노선: 인천 지하철 1호선'),
    HumanMessage(content='Using the provided information \n write 지연/사고 이유:'),
    AIMessage(

In [45]:
delay_template

"\n너는 항상 최선을 다하고 좋은 글을 작성해서 나를 기쁘게 해주고 있어.\n아래의 형식으로 작성해줘:\n\n{input}\n\n1) 제목과 본문으로 구분해서 출력해줘.\n2) 제목은 창의력 있고, 주목도 있게 구성해줘.\n3) 본문을 구성할 때는 Google과 Naver 검색의 검색엔진최적화(SEO)에 맞게 포스팅을 해줘\n4) 최대한 자세하게 작성해주고 신뢰도 있는 정보를 중심으로 포스팅을 해줘\n5) 뉴스기사를 이용해 블로그로 작성한 글과 마지막말은 대학생 블로그 글 서포터즈가 쓴 글 처럼 통통튀고 발랄한 말투로 작성해줘\n6) 공감하는 말을 추가해줘 ex) 지하철 파업으로 아침마다 출근하기 힘드네요\n7) 지연/사고 일시와 지연/사고 노선은 단답형으로 작성해줘. \n\n템플릿:\n{{여기에 블로그 제목}}\n{{시작하는 말}}\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!\n\n{{지연/사고 일시}}\n{{지연/사고 노선}}\n{{지연/사고 이유}}\n{{문의 사항 링크}}\n\n{{뉴스기사를 이용해 블로그로 작성한 글}}\n\n{{마무리 말}}\n오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요. \n'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.\n\n🔽 지하철 온다 소개 보러가기\nhttps://blog.naver.com/subway__onda/223258646349\n"

In [42]:
print(result["text"])

**영웅들이 함께 만든 기적, 인천 지하철의 감동적인 이야기**

안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

**지연/사고 일시**
2024년 5월 28일 오후 10시 09분

**지연/사고 노선**
인천 1호선

**지연/사고 이유**
응급 환자 발생

**문의 사항 링크**
[인천교통공사 고객센터](https://www.ictr.or.kr/kr/customerService/customerService)

---

안녕하세요, 지하철 이용자 여러분! 오늘은 인천 지하철에서 일어난 감동적인 소식을 전해드리려고 해요. 바로 우리 모두가 히어로가 될 수 있는 순간을 보여준 이야기랍니다.

2024년 5월 28일 저녁 10시 09분, 인천 1호선을 타고 가던 중 한 20대 승객이 갑작스러운 응급 상황에 처하게 되었어요. 이때 승객들은 모두가 한마음이 되어 신속하게 대처했답니다. 특히 현장에 있던 한 간호사분이 바로 심폐소생술을 시작했고, 인천교통공사 직원들이 AED(자동 심장 충격기)를 사용하여 환자의 생명을 구했어요.

이 덕분에 환자는 무사히 병원으로 이송될 수 있었고, 전철은 13분 정도 지연되었지만, 아무도 불평하지 않았어요. 오히려 모든 승객들이 한마음으로 응원하고 지지하는 모습이 정말 인상적이었어요. 인천교통공사에서도 이번 일에 도움을 준 시민에게 감사의 뜻을 전할 예정이라고 해요.

이 사건을 통해 다시 한번 느낀 것은, 우리 모두가 조금만 관심을 가지고 배려하면 큰 기적을 만들 수 있다는 점이에요. 지하철을 이용하는 동안, 우리 주변에 무슨 일이 일어나고 있는지 한 번 더 살펴보는 것도 중요한 것 같아요.

---

오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요.
'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기
[지하철 온다 블로그](https://blog.nave

In [22]:
train

[[{'question': 'Using the provided information \n write 지연/사고 일시:',
   'chat_history': [SystemMessage(content='The human provides an article about a subway delay caused by an air conditioner malfunction on Line 1, affecting many commuters. The AI summarizes the event, stating the train from Cheonan to Cheongnyangni stopped twice, causing significant discomfort and frustration among passengers. The train conductor announced the delay and the need for inspection over the PA system. The AI then offers tips for mitigating such inconveniences, like checking real-time updates, considering alternative transport, and establishing emergency contact networks. The AI expresses empathy for the affected passengers and hopes for improved subway conditions in the future.')],
   'text': '지연/사고 일시: 2023년 10월 15일 오전 8시 30분경'},
  {'question': 'Using the provided information \n write 지연/사고 노선:',
   'chat_history': [SystemMessage(content='The human provides an article about a subway delay caused by an air 

In [23]:
train_feedback

[[[{'question': '블로그 글 작성해줘',
    'chat_history': [SystemMessage(content='The human provides an article about a subway delay caused by an air conditioner malfunction on Line 1, affecting many commuters. The AI summarizes the event, stating the train from Cheonan to Cheongnyangni stopped twice, causing significant discomfort and frustration among passengers. The train conductor announced the delay and the need for inspection over the PA system. The AI then offers tips for mitigating such inconveniences, like checking real-time updates, considering alternative transport, and establishing emergency contact networks. The AI expresses empathy for the affected passengers and hopes for improved subway conditions in the future. The human then asks the AI to write the date and time of the delay/incident in Korean, and the AI responds with "지연/사고 일시: 2023년 10월 15일 오전 8시 30분경."'),
     HumanMessage(content='Using the provided information \n write 지연/사고 노선:'),
     AIMessage(content='지연/사고 노선: 1호선

In [25]:
for i in train_result:
    print(i["text"])

### 여기에 블로그 제목

#### 시작하는 말
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

#### 지연/사고 일시
2024년 5월 31일 오전 6시 46분

#### 지연/사고 노선
1호선 (천안 ~ 청량리 구간)

#### 지연/사고 이유
에어컨 고장으로 인한 차량 점검

#### 문의 사항 링크
🔽 문의 사항은 여기를 클릭해 주세요
https://blog.naver.com/subway__onda/223258646349

#### 뉴스기사를 이용해 블로그로 작성한 글
오늘 아침, 출근길에 겪은 불편함에 대해 이야기해볼게요. 2024년 5월 31일, 천안에서 청량리로 향하는 1호선 열차가 에어컨 고장으로 인해 두 번이나 멈춰 섰어요. 이로 인해 많은 승객 분들께서 큰 불편을 겪으셨을 거예요. 특히 아침 출근 시간대라 더더욱 힘드셨을 텐데요.

열차는 오전 6시 46분경 첫 번째로 멈춰 섰고, 그 후 차량 점검을 위해 다시 한 번 멈추게 되었어요. 열차 내에 있던 승객들은 덥고 답답한 환경 속에서 상당한 불편을 겪으셨을 거예요. 열차 안내 방송에서는 차량 점검을 위해 잠시 멈추게 되었다는 내용이 전달되었어요.

이럴 때를 대비해 몇 가지 팁을 공유드릴게요. 첫째, 실시간 지하철 정보를 확인할 수 있는 앱을 미리 설치해 두세요. 둘째, 대체 교통수단을 미리 알아두세요. 셋째, 비상 연락망을 구축해 두는 것도 좋은 방법이에요.

비록 이번 사건이 많은 불편을 초래했지만, 앞으로는 더 나은 지하철 서비스가 제공되기를 바랍니다. 지하철 온다의 오지통은 항상 여러분의 편안하고 안전한 출퇴근을 위해 노력할게요.

#### 마무리 말
오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개 보러가기
https://blog.naver.com/subway__

# 6월첫째주 task : 기사 4개 작성

# 지하철 지연 결과 -2개

In [27]:
train[2]

[{'question': 'Using the provided information \n write 지연/사고 일시:',
  'chat_history': [SystemMessage(content="The human provided an article about a 20-year-old emergency patient on Incheon Subway Line 1 who was saved by quick actions from citizens and the Incheon Transportation Corporation staff. The AI summarized the following events: Incheon Transportation Corporation's control room was alerted about the emergency, and a nearby nurse performed CPR. Station staff used an AED and continuously monitored the patient until emergency services arrived. The patient was successfully transported to the hospital, despite a 13-minute train delay. The AI highlighted that AEDs are available at all stations on lines 1, 2, and 7 and emphasized the importance of public cooperation in emergencies. The Incheon Transportation Corporation plans to thank the helpful citizen formally.")],
  'text': '지연/사고 일시: Incheon Subway Line 1에서 발생한 응급상황으로 인해 열차가 13분 지연되었습니다. 사고는 20대 응급환자가 발생한 사건으로, 신속한 시민과 인천교통공사 직원들의 

In [26]:
train_feedback[2]

[[{'question': '블로그 글 작성해줘',
   'chat_history': [SystemMessage(content='The human provided an article about a 20-year-old emergency patient on Incheon Subway Line 1 who was saved by quick actions from citizens and the Incheon Transportation Corporation staff. The AI summarized the following events: Incheon Transportation Corporation\'s control room was alerted about the emergency, and a nearby nurse performed CPR. Station staff used an AED and continuously monitored the patient until emergency services arrived. The patient was successfully transported to the hospital, despite a 13-minute train delay. The AI highlighted that AEDs are available at all stations on lines 1, 2, and 7 and emphasized the importance of public cooperation in emergencies. The Incheon Transportation Corporation plans to thank the helpful citizen formally. The human then asked the AI to write the incident details in Korean, and the AI provided a detailed description, reaffirming the importance of public cooperatio

In [28]:
train_result[2]

{'question': "\n너는 항상 최선을 다하고 좋은 글을 작성해서 나를 기쁘게 해주고 있어.\n아래의 형식으로 작성해줘:\n\n{input}\n\n1) 제목과 본문으로 구분해서 출력해줘.\n2) 제목은 창의력 있고, 주목도 있게 구성해줘.\n3) 본문을 구성할 때는 Google과 Naver 검색의 검색엔진최적화(SEO)에 맞게 포스팅을 해줘\n4) 최대한 자세하게 작성해주고 신뢰도 있는 정보를 중심으로 포스팅을 해줘\n5) 뉴스기사를 이용해 블로그로 작성한 글과 마지막말은 대학생 블로그 글 서포터즈가 쓴 글 처럼 통통튀고 발랄한 말투로 작성해줘\n6) 공감하는 말을 추가해줘 ex) 지하철 파업으로 아침마다 출근하기 힘드네요\n7) 지연/사고 일시와 지연/사고 노선은 단답형으로 작성해줘. \n\n템플릿:\n{{여기에 블로그 제목}}\n{{시작하는 말}}\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!\n\n{{지연/사고 일시}}\n{{지연/사고 노선}}\n{{지연/사고 이유}}\n{{문의 사항 링크}}\n\n{{뉴스기사를 이용해 블로그로 작성한 글}}\n\n{{마무리 말}}\n오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요. \n'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.\n\n🔽 지하철 온다 소개 보러가기\nhttps://blog.naver.com/subway__onda/223258646349\n",
 'chat_history': [SystemMessage(content='The human provided an article about a 20-year-old emergency patient on Incheon Subway Line 1 who was saved by quick actions from citizens and the Incheon Transportation Corporation staff.

In [103]:
for i in train_result:
    print(i["text"],end="\n---------------------------------\n")

### 사고로 인한 지하철 지연, 어떻게 확인하고 대처해야 할까요?

안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

#### 지연/사고 일시
2023년 2월 10일

#### 지연/사고 노선
서울 지하철 4호선

#### 지연/사고 이유
70대 남성 승객의 트랙 내 사고로 인한 일시적 운행 중단

#### 문의 사항 링크
[서울교통공사 지연증명서 발급](https://www.seoulmetro.co.kr/kr/page.do?menuIdx=406)

서울 지하철 4호선에서 2023년 2월 10일에 발생한 사고로 인해 많은 분들이 출근길에 불편을 겪으셨을 거예요. 당시 70대 남성 승객 한 분이 트랙 내에서 의식을 잃고 발견되어, 긴급 구조와 안전 조치로 인해 운행이 일시적으로 중단되었습니다. 이런 상황에서는 어떻게 대처해야 할까요? 오늘은 지하철 지연증명서 발급 방법과 함께 알아보겠습니다.

### 지하철 지연증명서 발급 방법

1. **서울교통공사 웹사이트 이용**: 서울교통공사 공식 웹사이트에 접속하여 '지연증명서 발급' 메뉴를 선택하세요. 사고 발생일과 시간, 노선을 입력하면 지연증명서를 발급받을 수 있어요.
   
2. **모바일 앱 사용**: '지하철 온다' 앱을 통해서도 지연증명서를 쉽게 발급받을 수 있어요. 앱 내에서 '지연증명서' 메뉴를 선택하고 필요한 정보를 입력하면 됩니다.

3. **역무원 문의**: 지연이 발생한 역의 역무원에게 직접 문의하여 지연증명서를 발급받을 수도 있어요. 역무원이 직접 발급해 주시기 때문에 바로 받아볼 수 있습니다.

이렇게 간단한 방법으로 지연증명서를 발급받을 수 있으니, 만약 출근이나 중요한 약속에 늦으셨다면 꼭 활용해보세요.

### 마무리 말
오지통이 실시간으로 다양한 지하철 정보를 업데이트할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.

🔽 지하철 온다 소개

In [19]:
print(source[0])

안산역 70대男 쓰러진 채 발견→4호선 한때 중단... 간편지연증명서 발급 방법·기준은?
 주지영 기자  승인 2023.02.10 09:32
 
오늘(10일) 안산역 선로에서 발생한 사상사고로 출근 시간대 4호선 열차 운행에 차질이 발생했다. (해당사진은 기사 내용과 무관합니다.) 사진=연합뉴스
오늘(10일) 안산역 선로에서 발생한 사상사고로 출근 시간대 4호선 열차 운행에 차질이 발생했다. (해당사진은 기사 내용과 무관합니다.) 사진=연합뉴스
[핀포인트뉴스 주지영 기자] 오늘(10일) 안산역 선로에서 발생한 사상사고로 출근 시간대 4호선 열차 운행에 차질이 발생했다.

이날 오전 6시 58분쯤 경기 안산시 단원구 원곡동 지하철 4호선 안산역 선로에서 70대 남성 A씨가 쓰러진 채 발견됐다. 

A씨는 4호선 서울방향 전동차 하부에서 심정지 상태로 발견됐다. A씨는 소방당국에 의해 인근 병원으로 옮겨졌다. 

이날 현장감식 등 사고 수습을 위해 4호선 양방향 열차 운행이 잠시 중단됐다. 이후 이날 오전 8시 37분께 안산~오이도역 간 상·하선 전구간 열차운행이 재개됐다.

그런 가운데  갑작스러운 지하철 지연 운행으로 출근길 시민들이 불편을 겪으면서 지하철 지연 증명서 발급 방법에 관심이 모아진다. 

지하철 지연 증명서는 지하철 연착으로 회사, 학교에 늦었을 경우 지각 사유를 뒷받침 할 수 있는 증명서다. 지연된 열차 기준 탑승 3일 이내에 발급 받을 수 있다.

서울교통공사의 여객운송약관에 따르면 지하철 지연 기준은 5분으로 5분 이상 지연이 발생했을 때 증명서 발급이 가능하다. 

KTX, 무궁화호 등은 지하철과 지연 기준이 다르다. 해당 열차를 운영하는 곳은 한국철동고사(코레일)로 이곳 약관에서는 20분 이상 지연이 발생해야 '열차 지연'으로 분류한다. 

다만 지연 사유가 내부 원인이어야 하며 천재지변, 테러 등은 지연 기준이 아니다.

열차 지연 시간은 서울교통공사, 코레일 홈페이지의 간편지연증명서 발급 서비스에서 확인할 수 있다. 


# 지하철 파업 결과 -2개

In [131]:
train[2:]

[[{'question': 'Using the provided information \n write 파업 일시:',
   'chat_history': [SystemMessage(content="The human provided an article about a railway union strike in South Korea, which started on September 14 and is expected to last until September 18. The strike has led to a significant reduction in the operation of passenger and freight trains, causing inconvenience for commuters and travelers. The union is demanding an expansion of public rail services, a full implementation of the 4-group 2-shift system, and KTX operations at Suseo Station. However, Korail, the railway operator, states that some demands are government policy issues and not subject to labor negotiations. The AI, named 오지통, summarized this information and offered tips on alternative transportation options. \n\nNew lines of conversation:\nHuman: Thank you for the detailed summary, 오지통. Could you also provide some specific advice for commuters who might be affected by the strike, like what alternative routes or tra

In [130]:
train_feedback[2:]

[[[{'question': '블로그 글 작성해줘',
    'chat_history': [SystemMessage(content='The human provided an article about a railway union strike in South Korea, which started on September 14 and is expected to last until September 18. The strike has led to a significant reduction in train services, causing inconvenience for many. The union\'s demands include an expansion of public rail services and changes to work shifts, while Korail states some demands are government policy issues. The AI summarized the article and provided tips on alternative transportation methods. The human asked for additional specific advice for commuters, and the AI suggested options like increased bus services, carpooling, bicycles, telecommuting, and checking operational metro lines. The human then asked the AI to write the strike dates in Korean, and the AI provided the information: "파업 일시: 2023년 9월 14일부터 9월 18일까지."'),
     HumanMessage(content='Using the provided information \n write 파업 노선:'),
     AIMessage(content='파

In [129]:
train_result[2:]

[{'question': "\n너는 지금부터 블로그 포스팅 전문 콘텐츠 마케터야.\n너는 항상 최선을 다하고 좋은 글을 작성해서 나를 기쁘게 해주고 있어.\n아래의 형식으로 작성해줘:\n\n{input}\n\n1) 제목과 본문으로 구분해서 출력해줘.\n2) 제목은 창의력 있고, 주목도 있게 구성해줘.\n3) 본문을 구성할 때는 Google과 Naver 검색의 검색엔진최적화(SEO)에 맞게 포스팅을 해줘\n4) 최대한 자세하게 작성해주고 신뢰도 있는 정보를 중심으로 포스팅을 해줘\n5) 글의 길이는 기본적으로 A4용지 1장 길이로 작성해줘\n6) 간단 요약부분과 마지막말은 대학생 블로그 글 서포터즈가 쓴 글 처럼 통통튀고 발랄한 말투로 작성해줘\n7) 공감하는 말을 추가해줘 ex) 지하철 파업으로 아침마다 출근하기 힘드네요\n8)파업 일시, 파업 노선은 단답형으로 작성해줘.\n\n템플릿:\n{{여기에 블로그 제목}}\n{{시작하는 말}}\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!\n\n{{파업 일시}}\n{{파업 노선}}\n{{파업 이유}}\n{{문의 사항 링크}}\n\n{{위 내용을 바탕으로 작성한 블로그 글}}\n\n{{마무리 말}}\n오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요. \n'지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철 역의 실시간 정보를 제공합니다.\n\n🔽 지하철 온다 소개 보러가기\nhttps://blog.naver.com/subway__onda/223258646349\n",
  'chat_history': [SystemMessage(content='The human provided an article about the South Korean railway union strike, detailing its impact on train services and commuters, and asked the AI to ve

In [126]:
for i in train_result[2:]:
    print(i["text"],end="\n\n-----------------------------------------------------------------------------------------------\n\n")

### 블로그 포스팅 전문 콘텐츠 마케터 버전

#### 제목
지하철 파업, 걱정하지 마세요! 대체 교통수단 총정리!

#### 본문
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

#### 파업 일시
2023년 9월 14일 ~ 2023년 9월 18일

#### 파업 노선
KTX, 새마을호, 무궁화호, 화물열차, 수도권 지하철

#### 파업 이유
- 공공철도 서비스 확대
- 4조 2교대제 전면 시행
- 수서역 KTX 운행

#### 문의 사항 링크
[코레일 고객센터](https://www.letskorail.com/ebizprd/EbizPrdMain.do)

#### 블로그 글
최근 철도 노조 파업 소식이 많은 사람들에게 큰 불편을 주고 있어요. 이번 파업은 2023년 9월 14일부터 9월 18일까지 예정되어 있어요. 파업으로 인해 KTX, 새마을호, 무궁화호 등의 열차 운행이 대폭 축소되고, 수도권 지하철도 83% 수준으로 운행된다고 해요. 출근길에 불편함을 겪고 있는 여러분, 정말 힘드시죠? 하지만 걱정하지 마세요! 오늘은 대체 교통수단에 대해 자세히 안내해 드릴게요.

### 파업 영향
이번 파업으로 인해 KTX는 평소의 76.4% 수준으로 운행되고, 새마을호와 무궁화호는 68.1%로 운행된다고 해요. 특히 화물열차는 26.3% 수준으로 줄어들어 물류에도 큰 영향이 있을 거예요. 수도권 지하철은 83% 수준으로 운행되어 출퇴근 시간에 지연이 발생할 수 있어요.

### 대체 교통수단
파업 기간 동안 불편을 최소화하기 위해 다양한 대체 교통수단을 이용해 보세요.
- **버스**: 경기도는 11개 노선에 98대의 예비 버스를 투입하고, 필요 시 전세 버스를 운영할 예정이에요. 인천은 출퇴근 시간대에 59대의 광역버스와 18대의 시내버스를 추가로 투입할 예정이에요.
- **카풀**: 이웃이나 동료들과 카풀을 고려해 보세요. 
- **자전거**: 단거리 이동 시 자전거를 이용해 보는 것도 좋은 방법이에

In [128]:
print(source[1])


“걱정돼서 일찍 나왔다”…지하철 경고 파업 첫날, 퇴근길 혼잡도↑
2023.11.09 19:11 입력 2023.11.10 11:04 수정
임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 첫날인 9일 서울 지하철 5호선 광화문역이 퇴근하는 시민들로 붐비고 있다. 2023.11.09. 권도현 기자
임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 첫날인 9일 서울 지하철 5호선 광화문역이 퇴근하는 시민들로 붐비고 있다. 2023.11.09. 권도현 기자

“교통공사 노조 파업으로 인해 열차 운행 지연될 수 있습니다. 장시간 지연 시 다른 교통수단을 이용해 주시기 바랍니다.”

서울 지하철 1~8호선을 운영하는 서울교통공사 노조의 경고 파업이 시작된 9일, 서울 전역 지하철 역사에서 이같은 안내 방송이 종일 흘러나왔다. 지하철 개찰구와 승강장 곳곳에는 “안전인력 감축, 시민·노동자 안전은 누가 지킵니까” “시민 부담 전가, 노동 안전 파괴, 즉각 중단하라” 등이 적힌 노조 홍보물이 붙었다. 오후 6시쯤 5호선 서대문역에서 만난 원상훈씨(55)는 “지하철 파업 소식을 듣고 회사에서 일찍 출발했다”면서 “걱정돼서 전철역까지 7분 거리를 내내 뛰었다”고 말했다.

서울교통공사의 민주노총 소속 노동조합이 이날부터 이틀간 경고파업에 돌입하면서 첫날 퇴근길에 일부 혼잡이 발생했다. 오전 7~9시 출근 시간대에는 지하철이 정상 운행됐으나 퇴근 시간인 오후 6시 이후에는 배차 간격이 다소 늘거나 평소 수준으로 유지됐다. 일찍 퇴근길에 나선 직장인들이 오후 5시부터 몰리면서 역사 내부가 비교적 이른 시간에 빽빽이 들어찼다. 전철을 피해 버스를 타려는 시민들로 버스 정류장에 긴 대기줄이 이어지기도 했다.

임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 이틀간 경고파업에 들어간 9일 5호선 광화문역에 관련 내용이 적힌 포스터가 붙어 있다. 2023.11.09 성동훈 기자
임금·단체협약 협상 결렬 여파로 서울교통공사 노조가 